Fibonacci Code of all language


Python – Fibonacci

In [ ]:
import time
import psutil
import os

def fibonacci_recursive(n):
    if n <= 1:
        return n
    return fibonacci_recursive(n-1) + fibonacci_recursive(n-2)

def main():
    n = 35
    
    # Measure initial memory
    process = psutil.Process(os.getpid())
    mem_before = process.memory_info().rss / 1024 / 1024  # MB
    
    # Compute Fibonacci with naive recursion
    start_time = time.time()
    start_cpu = time.process_time()
    result = fibonacci_recursive(n)
    end_cpu = time.process_time()
    end_time = time.time()
    
    # Measure final memory
    mem_after = process.memory_info().rss / 1024 / 1024  # MB
    max_memory = max(mem_before, mem_after)
    
    # Output metrics
    print(f"Execution time: {end_time - start_time:.2f} seconds")
    print(f"CPU time: {end_cpu - start_cpu:.2f} seconds")
    print(f"Max memory: {max_memory:.2f} MB")
    print(f"Fibonacci({n}) = {result}")

if __name__ == "__main__":
    main()

Java – Fibonacci

In [ ]:

import java.lang.management.*;

public class Fibonacci {
    public static long fibonacciRecursive(int n) {
        if (n <= 1) return n;
        return fibonacciRecursive(n-1) + fibonacciRecursive(n-2);
    }
    
    public static void main(String[] args) {
        int n = 35;
        
        // Measure initial memory
        Runtime runtime = Runtime.getRuntime();
        MemoryMXBean memoryBean = ManagementFactory.getMemoryMXBean();
        runtime.gc();
        long mem_before = runtime.totalMemory() - runtime.freeMemory();
        long non_heap_before = memoryBean.getNonHeapMemoryUsage().getUsed();
        
        // Compute Fibonacci with naive recursion
        ThreadMXBean bean = ManagementFactory.getThreadMXBean();
        long start_cpu = bean.isCurrentThreadCpuTimeSupported() ? bean.getCurrentThreadCpuTime() : 0;
        long start_time = System.nanoTime();
        long result = fibonacciRecursive(n);
        long end_time = System.nanoTime();
        long end_cpu = bean.isCurrentThreadCpuTimeSupported() ? bean.getCurrentThreadCpuTime() : 0;
        
        // Measure final memory
        runtime.gc();
        long mem_after = runtime.totalMemory() - runtime.freeMemory();
        long non_heap_after = memoryBean.getNonHeapMemoryUsage().getUsed();
        
        // Calculate memory usage
        long heap_memory = Math.max(mem_before, mem_after) / (1024 * 1024); // Heap in MB
        long non_heap_memory = Math.max(non_heap_before, non_heap_after) / (1024 * 1024); // Non-heap in MB
        long total_memory = (Math.max(mem_before, mem_after) + Math.max(non_heap_before, non_heap_after)) / (1024 * 1024); // Total in MB
        
        // Output metrics
        System.out.printf("Execution time: %.4f seconds\n", (end_time - start_time) / 1e9);
        System.out.printf("CPU time: %.4f seconds\n", (end_cpu - start_cpu) / 1e9);
        System.out.printf("Heap memory used: %d MB\n", heap_memory);
        System.out.printf("Non-heap memory used: %d MB\n", non_heap_memory);
        System.out.printf("Total memory used: %d MB\n", total_memory);
        System.out.println("Fibonacci(" + n + ") = " + result);
    }
}


Golang– Fibonacci

In [ ]:
package main

import (
	"fmt"
	"os"
	"runtime"
	"time"

	"github.com/shirou/gopsutil/v3/process"
)

// Pure recursive Fibonacci
func fibonacciRecursive(n int) int64 {
	if n <= 1 {
		return int64(n)
	}
	return fibonacciRecursive(n-1) + fibonacciRecursive(n-2)
}

// Warm-up to stabilize performance
func warmUp() {
	for i := 0; i < 100000; i++ {
		_ = 3 * 3
	}
	runtime.GC()
	time.Sleep(100 * time.Millisecond)
}

func measureMemory(p *process.Process, memChan chan float64, done chan struct{}) {
	for {
		select {
		case <-done:
			return
		default:
			var memStats runtime.MemStats
			runtime.ReadMemStats(&memStats)
			memMB := float64(memStats.Sys) / (1024.0 * 1024.0) // Use Sys for total memory
			select {
			case memChan <- memMB:
			default:
			}
			time.Sleep(10 * time.Millisecond) // Poll every 10ms
		}
	}
}

func main() {
	n := 35
	numRuns := 5 // Number of runs to average measurements

	// Warm-up CPU and runtime
	warmUp()

	// Prepare process info
	p, err := process.NewProcess(int32(os.Getpid()))
	if err != nil {
		fmt.Println("Error getting process info:", err)
		return
	}

	var totalWallTime, totalCPUTime float64
	var maxMemory float64

	for run := 0; run < numRuns; run++ {
		// Force GC and collect initial memory stats
		var memStats runtime.MemStats
		runtime.GC()
		runtime.Gosched()
		time.Sleep(100 * time.Millisecond) // Allow GC to finish
		runtime.ReadMemStats(&memStats)
		memBefore := float64(memStats.Sys) / (1024.0 * 1024.0) // in MB

		// Start memory polling
		memChan := make(chan float64, 100)
		done := make(chan struct{})
		go measureMemory(p, memChan, done)

		// Get CPU times before
		cpuBefore, err := p.Times()
		if err != nil {
			fmt.Println("Error getting CPU time:", err)
			return
		}

		// Get wall time before
		startWall := time.Now()

		// Execute the algorithm
		result := fibonacciRecursive(n)

		// Measure elapsed wall time
		elapsedWall := time.Since(startWall)

		// Get CPU times after
		runtime.Gosched()
		cpuAfter, err := p.Times()
		if err != nil {
			fmt.Println("Error getting CPU time:", err)
			return
		}
		cpuTime := (cpuAfter.User + cpuAfter.System) - (cpuBefore.User + cpuBefore.System)

		// Stop memory polling
		close(done)

		// Collect max memory from polling
		localMaxMemory := memBefore
		for len(memChan) > 0 {
			if mem := <-memChan; mem > localMaxMemory {
				localMaxMemory = mem
			}
		}

		// Read memory after execution
		runtime.GC()
		time.Sleep(100 * time.Millisecond)
		runtime.ReadMemStats(&memStats)
		memAfter := float64(memStats.Sys) / (1024.0 * 1024.0)
		if memAfter > localMaxMemory {
			localMaxMemory = memAfter
		}

		// Update max memory across runs
		if localMaxMemory > maxMemory {
			maxMemory = localMaxMemory
		}

		// Accumulate times
		totalWallTime += elapsedWall.Seconds()
		totalCPUTime += cpuTime

		// Output result for this run
		fmt.Printf("Run %d: Fibonacci(%d) = %d\n", run+1, n, result)
	}

	// Calculate averages
	avgWallTime := totalWallTime / float64(numRuns)
	avgCPUTime := totalCPUTime / float64(numRuns)

	// Output benchmark results
	fmt.Printf("\nAverage Execution time (wall): %.6f seconds\n", avgWallTime)
	fmt.Printf("Average CPU time: %.6f seconds\n", avgCPUTime)
	fmt.Printf("Max memory used: %.2f MB\n", maxMemory)
}


C++ – Fibonacci

In [ ]:
#include <iostream>
#include <chrono>
#include <windows.h>
#include <psapi.h>

long long fibonacci_recursive(int n) {
    if (n <= 1) return n;
    return fibonacci_recursive(n-1) + fibonacci_recursive(n-2);
}

int main() {
    const int n = 35;
    
    // Measure initial memory
    PROCESS_MEMORY_COUNTERS memInfo;
    GetProcessMemoryInfo(GetCurrentProcess(), &memInfo, sizeof(memInfo));
    double mem_before = memInfo.PeakWorkingSetSize / (1024.0 * 1024.0); // MB
    
    // Compute Fibonacci with naive recursion
    auto start = std::chrono::high_resolution_clock::now();
    clock_t start_cpu = clock();
    long long result = fibonacci_recursive(n);
    clock_t end_cpu = clock();
    auto end = std::chrono::high_resolution_clock::now();
    
    // Measure final memory
    GetProcessMemoryInfo(GetCurrentProcess(), &memInfo, sizeof(memInfo));
    double mem_after = memInfo.PeakWorkingSetSize / (1024.0 * 1024.0); // MB
    double max_memory = std::max(mem_before, mem_after);
    
    // Output metrics
    std::chrono::duration<double> diff = end - start;
    std::cout << "Execution time: " << diff.count() << " seconds\n";
    std::cout << "CPU time: " << (end_cpu - start_cpu) / (double)CLOCKS_PER_SEC << " seconds\n";
    std::cout << "Max memory: " << max_memory << " MB\n";
    std::cout << "Fibonacci(" << n << ") = " << result << "\n";
    return 0;
}